# Coffee Disease Classification using Transfer Learning with DenseNet on TensorFlow
In this notebook, we'll be implementing transfer learning for a coffee disease classification model using TensorFlow and DenseNet pretrained model.

## Install and import TensorFlow libraries
We will start by ensuring that we have the latest version of the TensorFlow package installed and importing the Tensorflow libraries we're going to use.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow
from  tensorflow import keras
print('TensorFlow version:',tensorflow.__version__)
print('Keras version:',keras.__version__)

TensorFlow version: 2.9.2
Keras version: 2.9.0


## Preprocess data and Prepare the base model

For us to use transfer learning, we will need a base model from which we can use the trained feature extraction layers. The DenseNet model is a CNN-based image classifier that has been pre-trained using a huge dataset of 3-color channel images of 224x224 pixels. Let's create an instance of it with some pretrained weights, excluding its final (top) prediction layer.

In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Load the data
data_dir = '/content/drive/MyDrive/Coffee Image Data Collection/'
batch_size = 32
img_height = 224
img_width = 224

print("Getting data...")
# Create a data generator
datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3)

# Load the data from the directory
print("Preparing training dataset...")
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

print("Preparing validation dataset...")
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

classnames = list(train_generator.class_indices.keys())
print("class names: ", classnames)

Getting data...
Preparing training dataset...
Found 31031 images belonging to 3 classes.
Preparing validation dataset...
Found 13297 images belonging to 3 classes.
class names:  ['Healthy', 'Leaf rust', 'Miner']


In [8]:
# Create a DenseNet model
model = keras.applications.DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(img_height, img_width, 3))

# Freeze the layers
model.trainable = False

# Add the final layers
x = model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
predictions = keras.layers.Dense(3, activation='softmax')(x)

# Compile the model
model = keras.models.Model(inputs=model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                           

In [9]:
# Create an early stopping callback
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps = validation_generator.samples // batch_size,
    callbacks=[early_stopping])

Epoch 1/20
878/969 [==========================>...] - ETA: 5:39 - loss: 0.0055 - accuracy: 0.9979

UnknownError: ignored